In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np
import keras
from keras import layers
import tensorflow as tf
from sklearn import preprocessing, model_selection
import random

QUALITY_THRESHOLD = 128
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = BATCH_SIZE * 2
num_classes = 6

2024-04-02 18:14:44.427500: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 18:14:44.427643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 18:14:44.564328: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
features = np.load("/kaggle/input/logsig3scaled/eeg_data_scaled.npy")
labels = np.load("/kaggle/input/logsig3scaled/targets.npy")

In [6]:
rows_without_nans = ~np.isnan(features).any(axis=1)

# Filter both features and labels to exclude rows with NaNs in features
features = features[rows_without_nans]
labels = labels[rows_without_nans]

In [ ]:
features.shape

In [ ]:
labels.shape

In [10]:
def percentage_between(matrix, lower_bound, upper_bound):
    """
    Calculate the percentage of points in a matrix that are between two values.
    
    Parameters:
    - matrix: NumPy array, the input matrix.
    - lower_bound: Lower bound of the range (exclusive).
    - upper_bound: Upper bound of the range (exclusive).
    
    Returns:
    - The percentage of points in the matrix that fall between the lower and upper bounds.
    """
    points_within_bounds = np.sum((matrix > lower_bound) & (matrix < upper_bound))
    total_points = np.size(matrix)
    percentage = (points_within_bounds / total_points) * 100
    return percentage


percentage_between(features, -1000, 1000)

100.0

In [12]:
np.min(features)

-243.12837086113217

In [13]:
shuffled_indices = np.random.permutation(len(features))
features = features[shuffled_indices]
labels = labels[shuffled_indices]

In [40]:
labels[1]

array([0.        , 0.        , 0.83333333, 0.        , 0.        ,
       0.16666667])

In [41]:
np.sum(np.isnan(labels))

0

In [14]:
from sklearn.model_selection import train_test_split

labels_argmax = np.argmax(labels, axis=1)

In [43]:
# vals_dict = {}
# for i in eeg["label"]:
#     if i in vals_dict.keys():
#         vals_dict[i] += 1
#     else:
#         vals_dict[i] = 1
# total = sum(vals_dict.values())

# # Formula used - Naive method where
# # weight = 1 - (no. of samples present / total no. of samples)
# # So more the samples, lower the weight

# weight_dict = {k: (1 - (v / total)) for k, v in vals_dict.items()}
# print(weight_dict)

In [15]:
# X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.2, stratify=labels_argmax)
from tensorflow.keras.utils import to_categorical

labels_encoded = to_categorical(labels_argmax, num_classes=num_classes)

X_train, X_test, Y_train, Y_test = train_test_split(features, labels_encoded, test_size=0.2, stratify=labels_argmax)

In [16]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [17]:
features[0].shape

(1390,)

In [17]:
def plot_history_metrics(history: keras.callbacks.History):
    total_plots = len(history.history)
    cols = total_plots // 2

    rows = total_plots // cols

    if total_plots % cols != 0:
        rows += 1

    pos = range(1, total_plots + 1)
    plt.figure(figsize=(15, 10))
    for i, (key, value) in enumerate(history.history.items()):
        plt.subplot(rows, cols, pos[i])
        plt.plot(range(len(value)), value)
        plt.title(str(key))
    plt.show()

In [24]:
# def create_model():
#     input_layer = keras.Input(shape=(2470, 1))

#     x = layers.Conv1D(
#         filters=32, kernel_size=3, strides=2, activation="relu", padding="same"
#     )(input_layer)
#     x = layers.BatchNormalization()(x)

#     x = layers.Conv1D(
#         filters=64, kernel_size=3, strides=2, activation="relu", padding="same"
#     )(x)
#     x = layers.BatchNormalization()(x)

#     x = layers.Conv1D(
#         filters=128, kernel_size=5, strides=2, activation="relu", padding="same"
#     )(x)
#     x = layers.BatchNormalization()(x)

#     x = layers.Conv1D(
#         filters=256, kernel_size=5, strides=2, activation="relu", padding="same"
#     )(x)
#     x = layers.BatchNormalization()(x)

#     x = layers.Conv1D(
#         filters=512, kernel_size=7, strides=2, activation="relu", padding="same"
#     )(x)
#     x = layers.BatchNormalization()(x)

#     x = layers.Conv1D(
#         filters=1024,
#         kernel_size=7,
#         strides=2,
#         activation="relu",
#         padding="same",
#     )(x)
#     x = layers.BatchNormalization()(x)

#     x = layers.Dropout(0.2)(x)

#     x = layers.Flatten()(x)

#     x = layers.Dense(4096, activation="relu")(x)
#     x = layers.Dropout(0.2)(x)

#     x = layers.Dense(
#         2048, activation="relu", kernel_regularizer=keras.regularizers.L2()
#     )(x)
#     x = layers.Dropout(0.2)(x)

#     x = layers.Dense(
#         1024, activation="relu", kernel_regularizer=keras.regularizers.L2()
#     )(x)
#     x = layers.Dropout(0.2)(x)
#     x = layers.Dense(
#         128, activation="relu", kernel_regularizer=keras.regularizers.L2()
#     )(x)
#     output_layer = layers.Dense(num_classes, activation="softmax")(x)

#     return keras.Model(inputs=input_layer, outputs=output_layer)

In [47]:
from tensorflow import keras
from tensorflow.keras import layers

def create_smaller_model():
    input_layer = keras.Input(shape=(2470, 1))

    # Convolutional layer without an activation argument
    x = layers.Conv1D(filters=16, kernel_size=3, strides=2, padding="same")(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.01)(x)  # Applying Leaky ReLU
    x = layers.Dropout(0.3)(x)

    x = layers.Conv1D(filters=32, kernel_size=3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.01)(x)  # Applying Leaky ReLU
    x = layers.Dropout(0.3)(x)

    x = layers.Conv1D(filters=64, kernel_size=5, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.01)(x)  # Applying Leaky ReLU
    x = layers.Dropout(0.4)(x)

    x = layers.Flatten()(x)

    x = layers.Dense(1024)(x)
    x = layers.LeakyReLU(alpha=0.01)(x)  # Applying Leaky ReLU
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(512)(x)
    x = layers.LeakyReLU(alpha=0.01)(x)  # Applying Leaky ReLU
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(256)(x)
    x = layers.LeakyReLU(alpha=0.01)(x)  # Applying Leaky ReLU
    x = layers.Dropout(0.5)(x)

    output_layer = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(inputs=input_layer, outputs=output_layer)


In [43]:
!rm -rf "/kaggle/working/best_model_V2_epoch_02.keras"

In [48]:
conv_model = create_smaller_model()
conv_model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 2470, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_18 (Conv1D)              │ (None, 1235, 16)       │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 1235, 16)       │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 1235, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 1235, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_19 (Conv1D)              │ (None, 618, 32)        │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 618, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 618, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 618, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_20 (Conv1D)              │ (None, 309, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 309, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 309, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 309, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 19776)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 1024)           │    20,251,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 20,921,702 (79.81 MB)

 Trainable params: 20,921,478 (79.81 MB)

 Non-trainable params: 224 (896.00 B)

In [49]:
epochs = 30

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="best_model_V2_epoch.keras",  # Modified to include epoch
        save_best_only=True,
        monitor="val_loss"  # Changed to "val_loss" if you're monitoring validation loss
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.2,
        patience=2,
        min_lr=0.000001,
    ),
]

optimizer = keras.optimizers.Adam(amsgrad=True, learning_rate=0.0001)
loss = keras.losses.KLDivergence()

In [29]:
# conv_model.load_weights("/kaggle/working/best_model.keras")

In [50]:
conv_model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[
    ],
)

conv_model_history = conv_model.fit(
    train_dataset,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_dataset,
)

Epoch 1/30
  12/1335 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - loss: 2.1674 

W0000 00:00:1712084952.298886      85 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1335/1335 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1.9725

W0000 00:00:1712084979.024981      87 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1712084979.651903      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1712084981.141898      88 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1335/1335 ━━━━━━━━━━━━━━━━━━━━ 44s 23ms/step - loss: 1.9724 - val_loss: 1.7678 - learning_rate: 1.0000e-04
Epoch 2/30
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 60s 14ms/step - loss: 1.7738 - val_loss: 1.7442 - learning_rate: 1.0000e-04
Epoch 3/30
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 1.7399 - val_loss: 1.6741 - learning_rate: 1.0000e-04
Epoch 4/30
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 1.6842 - val_loss: 1.6140 - learning_rate: 1.0000e-04
Epoch 5/30
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 1.6436 - val_loss: 1.5769 - learning_rate: 1.0000e-04
Epoch 6/30
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 1.6052 - val_loss: 1.5423 - learning_rate: 1.0000e-04
Epoch 7/30
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 1.5742 - val_loss: 1.4918 - learning_rate: 1.0000e-04
Epoch 8/30
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 1.5418 - val_loss: 1.4360 - learning_rate: 1.0000e-04
Epoch 9/30
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 1.5

In [51]:
conv_model.load_weights("/kaggle/working/best_model_V2_epoch.keras")

In [52]:
epochs=100

conv_model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[
    ],
)

conv_model_history = conv_model.fit(
    train_dataset,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=test_dataset,
)

Epoch 1/100
  12/1335 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - loss: 1.0281 

W0000 00:00:1712085660.611378      88 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1335/1335 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.9927

W0000 00:00:1712085686.678141      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1712085687.156618      88 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1712085688.288277      88 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1335/1335 ━━━━━━━━━━━━━━━━━━━━ 39s 22ms/step - loss: 0.9927 - val_loss: 0.8916 - learning_rate: 1.0000e-04
Epoch 2/100
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 22s 14ms/step - loss: 0.9773 - val_loss: 0.8764 - learning_rate: 1.0000e-04
Epoch 3/100
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - loss: 0.9695 - val_loss: 0.8645 - learning_rate: 1.0000e-04
Epoch 4/100
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 0.9549 - val_loss: 0.8570 - learning_rate: 1.0000e-04
Epoch 5/100
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 0.9354 - val_loss: 0.8450 - learning_rate: 1.0000e-04
Epoch 6/100
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 0.9227 - val_loss: 0.8337 - learning_rate: 1.0000e-04
Epoch 7/100
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 0.9121 - val_loss: 0.8257 - learning_rate: 1.0000e-04
Epoch 8/100
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - loss: 0.8935 - val_loss: 0.8125 - learning_rate: 1.0000e-04
Epoch 9/100
1335/1335 ━━━━━━━━━━━━━━━━━━━━ 19s 14ms/step - l

KeyboardInterrupt: 

In [ ]:
plot_history_metrics(conv_model_history)

In [ ]:
loss, accuracy, auc, precision, recall = conv_model.evaluate(test_dataset)
print(f"Loss : {loss}")
print(f"Top 3 Categorical Accuracy : {accuracy}")
print(f"Area under the Curve (ROC) : {auc}")
print(f"Precision : {precision}")
print(f"Recall : {recall}")


def view_evaluated_eeg_plots(model):
    start_index = random.randint(10, len(eeg))
    end_index = start_index + 11
    data = eeg.loc[start_index:end_index, "raw_values"]
    data_array = [scaler.fit_transform(np.asarray(i).reshape(-1, 1)) for i in data]
    data_array = [np.asarray(data_array).astype(np.float32).reshape(-1, 512, 1)]
    original_labels = eeg.loc[start_index:end_index, "label"]
    predicted_labels = np.argmax(model.predict(data_array, verbose=0), axis=1)
    original_labels = [
        le.inverse_transform(np.array(label).reshape(-1))[0]
        for label in original_labels
    ]
    predicted_labels = [
        le.inverse_transform(np.array(label).reshape(-1))[0]
        for label in predicted_labels
    ]
    total_plots = 12
    cols = total_plots // 3
    rows = total_plots // cols
    if total_plots % cols != 0:
        rows += 1
    pos = range(1, total_plots + 1)
    fig = plt.figure(figsize=(20, 10))
    for i, (plot_data, og_label, pred_label) in enumerate(
        zip(data, original_labels, predicted_labels)
    ):
        plt.subplot(rows, cols, pos[i])
        plt.plot(plot_data)
        plt.title(f"Actual Label : {og_label}\nPredicted Label : {pred_label}")
        fig.subplots_adjust(hspace=0.5)
    plt.show()


view_evaluated_eeg_plots(conv_model)